In [1]:
import numpy as np
import pandas as pd
import pickle
import calendar
import matplotlib.pyplot as plt
import pprint
import seaborn as sns
from scipy.stats import truncnorm
np.random.seed(2)

In [2]:
def get_truncated_normal(num_examples, mean=0, sd=1, low=0, upp=10):
    X = truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
    return X.rvs(num_examples)

def random_sampling(mean=0, sd=1, low=0, upp=10):
    return np.clip(np.random.randn(1)*sd/10 + mean , a_min=low, a_max = upp)
def calculate_wind_speed(height, a_0,a_1, a_2, a_3):
    speed = a_0 + a_1*np.exp(-(((height-a_2)/a_3)**2))
    for i in range(height.shape[0]):
        for j in range(height.shape[1]):
            if speed[i][j] <= 0:
                if j == 3:
                    speed[i][j] = 4 + 0.1*np.random.randn()
                else:
                    speed[i][j] = 1 + 0.05*np.random.randn()
#     speed[speed < 0] = 1 + 0.05*np.random.randn(speed.shape[0],speed.shape[1])
    return speed

def get_layer_heights_v3(num_examples, num_layers, city_list):
    elevation_dict = {
        'hawaii':3.1,
        'oakland':1.3, 
        'san diego':1.7, 
        'tucson':2.1, 
        'flagstaff':2.2, 
        'tenerife':2.3, 
        'antofagasta':2.3
    }
    if num_layers == 1:
        layer_heights = [0.02]
    elif num_layers == 2:
        #layer_heights = [5,12]
        layer_heights = [0.02,12]
    else:
        layer_heights = [0.02,5,12,20] # 20m, 5km, 12km, 20km
        
    height_delta = 0.01 # 10m 
    height_sea_level = np.zeros((num_layers, num_examples))
    height_observer = np.zeros((num_layers, num_examples))
    for m in range(num_examples):
        for i in range(num_layers):
            if i == 0:
                height_delta = 0.01 # 10m 
                mn_obs = layer_heights[i] - height_delta # 10m
                mx_obs = layer_heights[i] + height_delta # 30m
                mn_sea = elevation_dict[city_list[m]] + mn_obs # 10m + elevation
                mx_sea = elevation_dict[city_list[m]] + mx_obs # 30m + elevation
#                 ht_sea = get_truncated_normal(1, layer_heights[i] + elevation_dict[city_list[m]], height_delta, mn_sea, mx_sea)
                ht_sea = random_sampling(layer_heights[i] + elevation_dict[city_list[m]], height_delta, mn_sea, mx_sea)
                ht_obs = ht_sea - elevation_dict[city_list[m]]
            else:
                height_delta = 0.25 # 250m
                mn_sea = layer_heights[i] - height_delta
                mx_sea = layer_heights[i] + height_delta
                mn_obs = mn_sea - elevation_dict[city_list[m]]
                mx_obs = mx_sea - elevation_dict[city_list[m]]
                ht_sea = random_sampling(layer_heights[i], height_delta, mn_sea, mx_sea)
#                 ht_sea = get_truncated_normal(1, layer_heights[i], height_delta, mn_sea, mx_sea)
                ht_obs = ht_sea - elevation_dict[city_list[m]]
            height_sea_level[i,m] = ht_sea[0]
            height_observer[i,m] = ht_obs[0]
    return height_sea_level.T, height_observer.T 

In [3]:
num_examples = 20
num_layers = 1
city = ['hawaii']*num_examples
ht_sea, ht_obs = get_layer_heights_v3(num_examples, num_layers, city)

In [4]:
def generate_wind_profile_v2(num_examples=10,num_layers= 4, city_name = ''):
    assert num_layers <=4,"Maximum number of layers is 4"
    
    max_layers = 4
    speeds = np.zeros((num_examples, max_layers))
    directions = np.zeros((num_examples, max_layers))
    heights_observer = np.zeros((num_examples, max_layers))
    heights_sea_level = np.zeros((num_examples, max_layers))
    num_coeff = 4
    
    month_list = [m.lower() for m in calendar.month_name[1:] ]
    with open('./wind_data/wind_profiles_pickle.pkl', 'rb') as f:
        coeff_dict = pickle.load(f)
    cities_list = coeff_dict.keys()
    if city_name == '':
        city = np.random.choice([*cities_list], num_examples)
    else:
        city = np.array([city_name]*num_examples)
    
    heights_sea, heights_obs = get_layer_heights_v3(num_examples, num_layers, city)
    months = np.random.choice(month_list,num_examples)
    cf = np.zeros((num_examples,num_coeff, num_layers))
    for i,mth in enumerate(months):
        eps = np.random.rand(num_layers)
        for j in range(num_coeff):
            delta = coeff_dict[city[i]][mth][f'delta_A_{str(j)}']
            a_j = coeff_dict[city[i]][mth][f'A_{str(j)}']
            a_j_sample = (np.random.rand(num_layers)*2-1)*delta + a_j
            cf[i][j] = a_j_sample
    coeff_array = cf.transpose(1,0,2)  

    speeds_calculated= calculate_wind_speed(heights_sea*1000, coeff_array[0], coeff_array[1], coeff_array[2], coeff_array[3])
    directions_calculated = np.random.rand(num_examples, num_layers)*360
    
    speeds[:,:num_layers] = speeds_calculated
    directions[:, :num_layers] = directions_calculated
    heights_sea_level[:,:num_layers] = heights_sea
    heights_observer[:,:num_layers] = heights_obs
    profile = np.hstack((speeds, directions))
    results = {
        'profile': profile,
        'heights_sea_level': heights_sea_level,
        'heights_observer': heights_observer,
        'cities': city,
        'months': months,
        'coeffs': cf
    }
    return results

In [5]:
def encode_coefficient_array(coeffs):
    return np.array2string(coeffs, precision=5, separator=',').replace(',\n ', ';')

def decode_coefficient_array(coeff_string):
    layer_strings = coeff_string[2:-2].split('];[')
    num_coeffs = len(layer_strings)
    num_layers = len(layer_strings[0].split(","))
    coeff = np.zeros((num_coeffs, num_layers))
    for i in range(num_coeffs):
        layer_arr = [float(l.strip()) for l in layer_strings[i].split(',')]
        coeff[i] = layer_arr
    return coeff

In [6]:
def savefile(data, filename='wind_profile_data.txt'):
    num_samples, num_layers_2 = data['profile'].shape
    num_layers = num_layers_2//2
    with open(filename, 'w') as f:
        for i, sample_height in enumerate(data['heights_observer']):
            height_string = ','.join([f'{x*1000:.5f}' for x in sample_height])
            wind_string = ""
            for layer in range(num_layers):
                wind_layer_string = str(data['profile'][i][layer])+','+str(data['profile'][i][num_layers+layer])
                wind_string+= wind_layer_string+";"
            city_location_string = f'{data["cities"][i]},{data["months"][i]}'
            f.writelines([height_string+'\n', wind_string[:-1]+'\n', city_location_string+'\n', encode_coefficient_array(data['coeffs'][i])+'\n'])

In [8]:
results_tst = generate_wind_profile_v2(num_examples, num_layers)
savefile(results_tst, 'wind_profile_test_data_test_layer_1_grid.txt')